# About This Notebook

This notebook was done on Kaggle and downloaded after. It contains code that we use to scrape the games data from the 10k to 20k rows of our list of games. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam60kgamesdata/gamesdata.csv


In [2]:
df = pd.read_csv('/kaggle/input/steam60kgamesdata/gamesdata.csv')

In [3]:
from bs4 import BeautifulSoup
import time
import requests
import json
from tqdm import tqdm

In [4]:
counter = 0
data = []

for counter in tqdm(range(20000,30000)):
        hlink = df['Link'][counter]
        response = requests.get(hlink)
        soup1 = BeautifulSoup(response.text, 'html.parser')

        description = soup1.find('div',id='game_area_description')
        description = description.text.strip().replace('\r','').replace('\t','').replace('\n','') if description else None

        reviews = soup1.find('div', class_='user_reviews_summary_row')
        reviews = reviews.text.strip().replace('\r','').replace('\t','').replace('\n','') if reviews else None
        tags_div = soup1.find('div', class_='glance_tags popular_tags')
        tags = tags_div.find_all('a', class_='app_tag') if tags_div else 'N/A'

        if tags != 'N/A':
            tagged = [tag.text.strip() for tag in tags]
            tagged_string = ', '.join(tagged)
        else:
            tagged_string = 'N/A'

       
        data.append([description, reviews, tagged_string])
        
        counter+=1
    

100%|██████████| 10000/10000 [3:02:17<00:00,  1.09s/it]


In [5]:
data_df = pd.DataFrame(data, columns=['description','reviews','tags'])
data_df

,description,reviews,tags
0,"About This GameIIn fact, the only thing a knig...",All Reviews:1 user reviews- Need more user rev...,"Runner, Automation, Building, Adventure, Funny..."
1,About This GameDestroy the Dummies is a simple...,All Reviews:3 user reviews- Need more user rev...,"1990's, Fighting, Difficult, Puzzle, Arcade, A..."
2,"About This GameIn this puzzle game, play as an...",All Reviews:Positive(17)- 100% of the 17 user ...,"Casual, Sokoban, Robots, Puzzle, 2D, Choices M..."
3,About This GameIt’s a puzzle game. Every level...,All Reviews:6 user reviews- Need more user rev...,"Puzzle, Puzzle Platformer, Casual, Logic, 3D, ..."
4,About This GameWelcome to Weed County. After t...,All Reviews:4 user reviews- Need more user rev...,"Indie, Adventure, RPG, Simulation, Gore, Viole..."
...,...,...,...
9995,About This GameThe battle for magic and techno...,All Reviews:2 user reviews- Need more user rev...,"Action, Early Access, Fantasy, Twin Stick Shoo..."
9996,About This GameIts a game!Take the time to app...,All Reviews:No user reviews,"Casual, Simulation, Action, Puzzle, Physics"
9997,About This GameVolticPistol is a 2D medieval-c...,All Reviews:Positive(10)- 100% of the 10 user ...,"Side Scroller, Action Roguelike, Bullet Hell, ..."
9998,"About This GameWhen Maria, a globe-trotting an...",Recent Reviews:Very Positive(142)- 90% of the ...,"Casual, Indie, Puzzle, Story Rich, Female Prot..."


In [6]:
data_df.to_csv('twenty_k_games.csv', index=False)